In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt


from timeit import default_timer
from Solver import *
from functools import partial

import torch

import sys
sys.path.append('../Utility')
from Numerics import gradient_first,  gradient_first_c2f, gradient_first_f2c, interpolate_c2f, interpolate_f2c, psi_fft_sol, gradient_fft
import NeuralNet
import PlotDefault



In [4]:
Nx = 10000

q_mean   = 5.0*np.random.random(Nx) * q_scale
dpv_mean = 5.0*np.random.random(Nx) * dpv_scale
psi_mean = 5.0*np.random.random(Nx) * psi_scale

mu_mean  = (0.1*np.random.random(Nx) + 1.0) * mu_scale

# scale input 
nn_save_name = "pretrain.nn"
x_train = torch.from_numpy(np.stack((np.fabs(q_mean).flatten()/q_scale, dpv_mean.flatten()/dpv_scale, np.fabs(psi_mean).flatten()/psi_scale)).T.astype(np.float32)) 
y_train = torch.from_numpy(mu_mean.flatten()[:,np.newaxis].astype(np.float32)) / mu_scale



ind = x_train.shape[1]
outd = y_train.shape[1] 

learning_rate = 0.001
step_size = 100
gamma = 0.5  
epochs = 1000
batch_size = 64


net = NeuralNet.FNN(ind, outd, layers, width, activation, initializer, outputlayer) 

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)

optimizer = NeuralNet.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = torch.nn.MSELoss(reduction='sum')
t0 = default_timer()
for ep in range(epochs):
    net.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:

        optimizer.zero_grad()
        out = net(x)
        
        loss = myloss(out , y)*100
        loss.backward()

        optimizer.step()
        train_l2 += loss.item()

#     scheduler.step()

    if ep % 100 == 0:
        # train_l2/= ntrain
        t2 = default_timer()
        print("Epoch : ", ep, " Epoch time : ", t2-t1, " Train L2 Loss : ", train_l2)
        if nn_save_name is not None:
            torch.save(net, nn_save_name)
        


Epoch :  0  Epoch time :  0.15994891477748752  Train L2 Loss :  4049017.8393554688
Epoch :  100  Epoch time :  0.100262519903481  Train L2 Loss :  862.3997733592987
Epoch :  200  Epoch time :  0.09387932764366269  Train L2 Loss :  854.7757195234299
Epoch :  300  Epoch time :  0.09599088691174984  Train L2 Loss :  854.1250422000885
Epoch :  400  Epoch time :  0.09634441370144486  Train L2 Loss :  864.3645453453064
Epoch :  500  Epoch time :  0.09134048502892256  Train L2 Loss :  856.7524404525757
Epoch :  600  Epoch time :  0.11628606915473938  Train L2 Loss :  857.8716133832932
Epoch :  700  Epoch time :  0.09332923218607903  Train L2 Loss :  857.1931837797165
Epoch :  800  Epoch time :  0.09523880109190941  Train L2 Loss :  856.0808984041214
Epoch :  900  Epoch time :  0.09304055524989963  Train L2 Loss :  855.1683585643768


In [6]:

# print(trained_net.modus['LinM{}'.format(1)].weight)
# print(trained_net.modus['LinM{}'.format(1)].bias)
print(net.modus['LinMout'].weight)
print(net.modus['LinMout'].bias)

Parameter containing:
tensor([[ 4.3011e-05,  3.5696e-04, -1.3163e-03]], requires_grad=True)
Parameter containing:
tensor([1.0479], requires_grad=True)
